In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SummaryIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
    load_index_from_storage,
    Settings
)
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-****"
Settings.llm = OpenAI(temperature=0, model="gpt-4o")
# service_context = ServiceContext.from_defaults(llm=llm)

In [ ]:
import pandas as pd

# Load the Excel sheet
docs_df = pd.read_excel("CLM_Agents_AutoGen.xlsx")

# Initialize dictionaries to hold the indices and query engines
indices = {}
summary_indices = {}
vector_query_engines = {}
summary_query_engines = {}

# Loop through each row in the DataFrame
for _, row in docs_df.iterrows():
    folder_name = row['folder_name']
    persist_dir = "./storage/" + folder_name#row['index_storage_path']

    # Define index names
    index_name = f"{folder_name}_vector"
    summary_index_name = f"{folder_name}_summary"

    try:
        # Load indices from storage
        indices[index_name] = load_index_from_storage(StorageContext.from_defaults(persist_dir=persist_dir + "_vector"))
        summary_indices[summary_index_name] = load_index_from_storage(StorageContext.from_defaults(persist_dir=persist_dir + "_summary"))

        # Create query engines
        vector_query_engines[index_name] = indices[index_name].as_query_engine(similarity_top_k=3)
        summary_query_engines[summary_index_name] = summary_indices[summary_index_name].as_query_engine()
        
    except Exception as e:
        print(f"Error loading indices for {folder_name}: {e}")

In [ ]:
from llama_index.agent.openai import OpenAIAgent

# Initialize dictionaries to hold agents
agents = {}

# Loop through each row in the DataFrame
for _, row in docs_df.iterrows():
    folder_name = row['folder_name']
    
    # Construct keys for the query engines
    vector_query_engine_key = f"vector_query_engines['{folder_name}_vector']"
    summary_query_engine_key = f"summary_query_engines['{folder_name}_summary']"

    try:
        # Define tools for the current document
        query_engine_tools = [
            QueryEngineTool(
                query_engine=eval(vector_query_engine_key),  # Access vector query engine
                metadata=ToolMetadata(
                    name=f"{folder_name}_vector_tool",
                    description=row['vector_description'],
                ),
            ),
            QueryEngineTool(
                query_engine=eval(summary_query_engine_key),  # Access summary query engine
                metadata=ToolMetadata(
                    name=f"{folder_name}_summary_tool",
                    description=row['summary_description'],
                ),
            ),
        ]

        # Build the agent
        function_llm = OpenAI(model="gpt-4o", temperature=0)
        agent = OpenAIAgent.from_tools(
            query_engine_tools,
            llm=function_llm,
            verbose=True,
        )

        # Store the agent
        agents[f"{folder_name}"] = agent

    except Exception as e:
        print(f"Error creating agent for {folder_name}: {e}")

In [ ]:
# Initialize list to hold all tools
all_tools = []

# Loop through each row in the DataFrame
for _, row in docs_df.iterrows():
    folder_name = row['folder_name']
    
    # Define the summary description
    top_level_description = row['top_level_description']

    # Define tool
    doc_tool = QueryEngineTool(
        query_engine=agents[f"{folder_name}"],
        metadata=ToolMetadata(
            name=f"tool_{folder_name}",
            description=top_level_description,
        ),
    )
    all_tools.append(doc_tool)

In [ ]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [ ]:
from llama_index.agent.openai import OpenAIAgent

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
You are an agent designed to answer queries about community led monitoring (CLM) from guidance from major organizations in the field.
Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

""",
    verbose=True,
)